In [ ]:
"""These are some task-specific bits of code used in general FITS handling and adding WCS. 
They are not related to each other and sometimes present different methods
of performing the same task."""

#HANDLING FITS TABLES

import numpy as np
from astropy.io import fits
from astropy.table import Table

#Opening a fits table file
file = fits.open("nameofthefile.fits") 
data = file[1].data 
#This is not table-type specific. 
#Number in the bracket specifies the extension's number, which is a table in this case.

#Get information about columns - their names, formats, units etc. 
cols = file[1].columns
cols.info() 

#Get the first row of the table.
print (data[0])

#This is a great tool! It creates table-looking element instead just arrays 
#of strings and numbers. Remember to import "Table" from astropy.io

table_corr = Table(file[1].data) 
table_corr #No "print" necessary 

#GETTING SPECIFIC INFORMATION FROM THE HEADER
#RA and DEC are just example of information one can get. Name must be exaclty like in the header

from astropy.io import fits

file = fits.open("example.fits")
ra = file[0].header['RA']

file = fits.open("example.fits")
dec = file[0].header['DEC']

#CONVERTING RA, DEC TO DEGREES

#Specify in the following format (string), keep as many decimals in seconds as you want
ra = hh:mm:ss...
dec = +deg:mm:ss...

degra=float(ra[0:2])*15+float(ra[3:5])/4+float((ra[6:]))/240
degdec=float(dec[1:3])+float(dec[4:6])/60+float((dec[7:]))/3600
print (degra, degdec)

"""Notice this would be more useful as a function, I had no time to see if it works, but it
would be something like:"""
def ra_to_degra(ra):
    degra=float(ra[0:2])*15+float(ra[3:5])/4+float((ra[6:]))/240
    return degra

#ADDING WCS USING ASTROPY WCS METHOD
"""For this method, you already need to know all the parameters. Getting paramteres of a WCS fit
is the challenge handled by extensive astrometric codes like Astrometry.net. 
This code only automatically add WCS info to headers and check for obvious bugs"""

import numpy as np
from astropy import wcs
from astropy.io import fits
import sys

#Create a new WCS object (think you're making an empty grid)
w = wcs.WCS(naxis=2)

"""These are just examples of how to bulid some of WCS information. The actual type of
information you will be appending depends on your parameters and parameter conventions. There
are many different ways of defining WCS parameters. For example, cdelt and cd are mutually
exclusive. Refer to FITS documentation for more info
https://fits.gsfc.nasa.gov/fits_wcs.html
The code below creates headers with conventional WCS keywords and input values"""

w.wcs.crpix = [759.075470, 716.825058] #Location of reference pixel
w.wcs.cdelt = np.array([-pixdeg, pixdeg], np.float_) # Sets vector properties
# Consider it as your "unit vector" in degrees/pixel
w.wcs.ctype = ["RA-TAN", "DEC-TAN"] #Type of projection
w.wcs.cd = [[-0.000115, 0.000001],[0.000001, 0.000115]] #Optional rotation matrix

"Most of this just checks for errors"
# Some pixel coordinates of interest.
pixcrd = np.array([[0, 0], [24, 38], [45, 98]], np.float_)

# Convert pixel coordinates to world coordinates
world = w.wcs_pix2world(pixcrd, 1) #1 or 0 in argument depending on pixel grid convention
print(world)

# Convert the same coordinates back to pixel coordinates.
pixcrd2 = w.wcs_world2pix(world, 1)
print(pixcrd2)

# These should be the same as the original pixel coordinates, modulo
# some floating-point error. There usually is one.
assert np.max(np.abs(pixcrd - pixcrd2)) < 1e-6

#ADDING WCS MANUALLY

"""Adding WCS manually by editing/adding the headers that specify WCS parameters.
As before, the parameters depend on you and the conventions and must be calculated before"""

from astropy.io import fits
import sys

data, header = fits.getdata("A9.fits", header=True)

"""These names needs to be exactly conventional, see FITS manual https://fits.gsfc.nasa.gov/fits_wcs.html"""

header["WCSAXES"] = 2
header["RADESYS"] = "ICRS"
header["CTYPE1"] = "RA-TAN"
header["CTYPE2"] = "DEC-TAN"
header["CRVAL1"] = 295.508216
header["CRVAL2"] = 40.278292
header["CRPIX1"] = 562.495956
header["CRPIX2"] = 196.985538
header["CD1_1"] = -0.000115
header["CD1_2"] = 0.000001
header["CD2_1"] = 0.000002
header["CD2_2"] = 0.000116
header['CUNIT1'] = 'deg' 
header['CUNIT2'] = 'deg' 

fits.writeto('A9newwcs.fits', data, header, overwrite=True) #Saves new  headers

#PRINTING HEADER + PIXEL TRANSFORMATIONS

"""This method allows to convert from pixel to world coordinates, given
the WCS information already exists within a file."""

from astropy.wcs import WCS
from astropy.io import fits

#Prints header
hdu_number = 0 # HDU - header data unit
print (fits.getheader("NGC6819E0000learning example.fits", hdu_number))

#Converts from pixel coords to world coords
w = WCS('NGC6819E0000learning example.fits')
lon, lat = w.all_pix2world(80, 273, 1) # Args: x pixl, y pixl, pixl grid convention
print(lon, lat)